### Motor Control Logic

##### Simplified Motor Control: 
This assumes direct GPIO control of DC motors for a basic rover. 
##### No Precise Distance: 
Obstacle proximity is inferred from bounding box positions without an actual depth sensor. 
##### Test Environment: 
Start in a controlled setting, and add complexity iteratively.

In [ ]:
import torch
import cv2
import RPi.GPIO as GPIO

# Parameters  
OBSTACLE_THRESHOLD = 0.50  
CRITICAL_ZONE_WIDTH = 0.6
PERSON_CLASS_ID = 0  
APPROACH_DISTANCE_THRESHOLD = 0.4  

# Model Setup
model = DetectMultiBackend('yolov5s.pt', device='cpu')  # Assuming CPU on Raspberry Pi 

# Camera Setup
cap = cv2.VideoCapture(0)  # Assumes Pi Camera with index 0

# Motor GPIO Setup
GPIO.setmode(GPIO.BOARD)  
LEFT_FORWARD_PIN = 12
LEFT_BACKWARD_PIN = 16
RIGHT_FORWARD_PIN = 18
RIGHT_BACKWARD_PIN = 22

GPIO.setup(LEFT_FORWARD_PIN, GPIO.OUT)
GPIO.setup(LEFT_BACKWARD_PIN, GPIO.OUT)
GPIO.setup(RIGHT_FORWARD_PIN, GPIO.OUT)
GPIO.setup(RIGHT_BACKWARD_PIN, GPIO.OUT)

# Initialize motor pins to stopped state
GPIO.output(LEFT_FORWARD_PIN, GPIO.LOW) 
GPIO.output(LEFT_BACKWARD_PIN, GPIO.LOW) 
GPIO.output(RIGHT_FORWARD_PIN, GPIO.LOW) 
GPIO.output(RIGHT_BACKWARD_PIN, GPIO.LOW) 


# Motor Control Functions 
def move_forward():
    GPIO.output(LEFT_FORWARD_PIN, GPIO.HIGH)
    GPIO.output(RIGHT_FORWARD_PIN, GPIO.HIGH)
    # Ensure other direction pins are LOW if necessary for your setup 

def turn_left():
    GPIO.output(LEFT_BACKWARD_PIN, GPIO.HIGH)  
    GPIO.output(RIGHT_FORWARD_PIN, GPIO.HIGH)

def turn_right():
    GPIO.output(LEFT_FORWARD_PIN, GPIO.HIGH)  
    GPIO.output(RIGHT_BACKWARD_PIN, GPIO.HIGH)

def stop():
    GPIO.output(LEFT_FORWARD_PIN, GPIO.LOW) 
    GPIO.output(LEFT_BACKWARD_PIN, GPIO.LOW) 
    GPIO.output(RIGHT_FORWARD_PIN, GPIO.LOW) 
    GPIO.output(RIGHT_BACKWARD_PIN, GPIO.LOW) 

def approach(): 
    # Need to implement the approach
    pass  

# Obstacle Detection Parameters
OBSTACLE_THRESHOLD = 0.50  # Above this, object is treated as obstacle
CRITICAL_ZONE_WIDTH = 0.6  # Percent of frame width the "critical zone" covers

# Main Loop
while True:
    ret, frame = cap.read()

    # YOLOv5 Inference
    pred = model(frame) 

    detection_found = False  # Track whether a person or obstacle is seen

    # Process Detections 
    for i, det in enumerate(pred):
        for *xyxy, conf, cls in reversed(det):
            if conf > OBSTACLE_THRESHOLD: 
                detection_found = True 
                x_center = (xyxy[0] + xyxy[2]) / 2 
                relative_center = x_center / frame.shape[1] 

                if names[int(cls)] == 'person': 
                    stop() 
                    # Distance Check (Optional)
                    if calculate_distance(xyxy) < APPROACH_DISTANCE_THRESHOLD:  
                        approach()

                else:  # General obstacle avoidance
                    stop() 
                    if relative_center < 0.5: 
                        turn_right() 
                    else: 
                        turn_left()                     
                    time.sleep(0.5)  # Turn duration logic 

    # Default: Move forward if nothing important  detected
    if not detection_found:
        move_forward()  

# Cleanup before exit
stop()
GPIO.cleanup()
cap.release()
